In [1]:
import baseline_ds

/home/arimo/Desktop/Studily/ANLP/Advanced_NLP_group_1/A3/baseline_ds.py:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/home/arimo/.local/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/42466 [00:00<?, ? examples/s]

Map:   0%|          | 0/5441 [00:00<?, ? examples/s]

Map:   0%|          | 0/5328 [00:00<?, ? examples/s]

In [2]:
import datasets

In [3]:
def label_to_num(text,labels):
    '''
    This function reverse int class label back to strings.
    '''
    lablist=[]
    for label,token in zip(labels,text):
        if label == 'V':
            label = '_'
            pred = [token]
        lablist.append(baseline_ds.label_dict[label])
    dict = {'tokens':text, 'srl_arg_tags':lablist, 'pred':pred}
    return dict

In [4]:
text = ['I','love','you','.']
lab = ['ARG0','V','ARG1','_']

text2 = ['I','hate','you','so','much','.']
lab2 = ['ARG0','V','ARG1','_','_','_']

In [5]:
testcase = [label_to_num(text,lab),label_to_num(text2,lab2)]

In [6]:
caseds = datasets.Dataset.from_list(testcase)
tokenized_case = caseds.map(baseline_ds.tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertConfig, BertModel, AutoModel, AutoTokenizer, BertForSequenceClassification, DataCollatorForTokenClassification, Trainer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
from transformers import AutoModelForTokenClassification, Trainer, AutoTokenizer, DataCollatorForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer)

model = AutoModelForTokenClassification.from_pretrained("dannashao/bert-base-uncased-finetuned-advanced-srl_arg",num_labels=len(baseline_ds.label_dict),id2label=baseline_ds.label_dict_rev, label2id=baseline_ds.label_dict)
base_trainer = Trainer(model,data_collator=data_collator,tokenizer=tokenizer)

config.json:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [14]:
pred,lab,_ = base_trainer.predict(tokenized_case)

In [21]:
import numpy as np
def remove_and_reverse(predictions, labels):
    pred = np.argmax(predictions, axis=2)
    predlists, lablists = [],[]
    for i in range(len(labels)):
        preds, labs = [],[]
        for j in range(len(labels[i])):
            if labels[i][j] != -100:
                preds.append(baseline_ds.label_dict_rev[pred[i][j]])
                labs.append(baseline_ds.label_dict_rev[labels[i][j]])
        preds.pop()
        labs.pop()
        predlists.append(preds)
        lablists.append(labs)
    return predlists, lablists

In [22]:
remove_and_reverse(pred, lab)

([['ARG0', '_', 'ARG1', '_'], ['ARG0', '_', 'ARG1', '_', 'ARGM-EXT', '_']],
 [['ARG0', '_', 'ARG1', '_'], ['ARG0', '_', 'ARG1', '_', '_', '_']])